In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI as Chat
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

load_dotenv()

True

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
model = os.getenv("OPENAI_MODEL", "gpt-4o")
temperature = float(os.getenv("OPENAI_TEMPERATURE", 0))

NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
llm = Chat(
    openai_api_key=openai_api_key,
    model=model,
    temperature=temperature
)

In [4]:
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [5]:
# Define the Cypher generation template
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Schema: {schema}
Question: {question}
"""

In [7]:
# Create a prompt template for Cypher generation
cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

print(cypher_generation_prompt)

input_variables=['question', 'schema'] input_types={} partial_variables={} template="\nYou are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.\nConvert the user's question based on the schema.\n\nSchema: {schema}\nQuestion: {question}\n"


In [10]:
# Initialize the GraphCypherQAChain with the language model, graph, and prompt template
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

print(cypher_chain)

verbose=True graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x1061d8440> cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template="\nYou are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.\nConvert the user's question based on the schema.\n\nSchema: {schema}\nQuestion: {question}\n"), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10c470ef0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10c472660>, root_client=<openai.OpenAI object at 0x10bdc2c90>, root_async_client=<openai.AsyncOpenAI object at 0x10c312f30>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}) qa_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['con

In [13]:
# Invoke the chain with a sample query and print the result
response = cypher_chain.invoke(
    {"query": "What is Climate Change?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Chunk)
WHERE c.text CONTAINS "climate change"
RETURN c.text AS RelevantText, c.source AS Source

Full Context:
[{'RelevantText': 'Climate change refers to long-term shifts in temperatures and weather patterns, primarily caused by human activities such as burning fossil fuels. The impact of climate change includes rising sea levels, increased frequency of extreme weather events, and biodiversity loss. Addressing climate change requires global cooperation, and actions like reducing carbon emissions and promoting sustainable practices are key to mitigation.', 'Source': '/Users/akileshjayakumar/Documents/neo4j-llm-fundamentals/rag_test_files/Climate_Change_and_Impact.txt'}]

> Finished chain.


In [14]:
print(response["result"])

Climate change refers to long-term shifts in temperatures and weather patterns, primarily caused by human activities such as burning fossil fuels.
